<a href="https://colab.research.google.com/github/jeanlucjackson/w266_final_project/blob/main/code/sandboxes/RR/rr_evaluation_sandbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
inference_root = "/content/drive/MyDrive/w266 NLP Final Project/Predictions/"
filename = inference_root + "evaluation_database.json"

# Read JSON into dataframe
evaluation_db_df = pd.read_json(filename)

In [25]:
evaluation_db_df.columns

Index(['model_checkpoint', 'target', 'prediction', 'rouge1', 'rouge2',
       'rougeL', 'rougeLsum', 'bleurt', 'bertscore-precision',
       'bertscore-recall', 'bertscore-f1', 'meteor', 'base_model', 'framework',
       'trained_on', 'tested_on', 'nickname'],
      dtype='object')

In [47]:
pd.set_option('precision', 4)

In [57]:
evaluation_db_df[(evaluation_db_df['base_model'] == 'T5_base_pt_long') &
                 (evaluation_db_df['trained_on'] == 'squad') &
                 (evaluation_db_df['tested_on'] == 'squad')
                 ].groupby(['base_model', 'trained_on', 'tested_on']).mean()

,,,rouge1,rouge2,rougeL,rougeLsum,bleurt,bertscore-precision,bertscore-recall,bertscore-f1,meteor
base_model,trained_on,tested_on,,,,,,,,,
T5_base_pt_long,squad,squad,0.4842,0.2684,0.4521,0.4521,-0.3187,0.8714,0.8598,0.8649,0.4555


In [53]:
evaluation_db_df[evaluation_db_df['tested_on'] == 'nq'].groupby(['base_model', 'trained_on', 'tested_on']).mean()

rouge1  rouge2  rougeL  rougeLsum  \
base_model        trained_on tested_on                                      
T5_base_pt_long   amalgam    nq         0.5459  0.3218  0.5202     0.5202   
                  nq         nq         0.5440  0.3213  0.5191     0.5191   
                  quac       nq         0.1491  0.0239  0.1448     0.1448   
                  squad      nq         0.4252  0.2081  0.4065     0.4065   
                  triviaqa   nq         0.3157  0.1314  0.2723     0.2723   
bart_base_pt_long amalgam    nq         0.5694  0.3531  0.5450     0.5450   
                  nq         nq         0.5764  0.3570  0.5504     0.5504   
                  quac       nq         0.3342  0.1369  0.3261     0.3261   
                  squad      nq         0.4288  0.2126  0.4106     0.4106   
                  triviaqa   nq         0.4025  0.1971  0.3656     0.3656   

                                        bleurt  bertscore-precision  \
base_model        trained_on tested_on                                
T5_base_pt_long   amalgam    nq        -0.3562               0.8601   
                  nq         nq        -0.3687               0.8616   
                  quac       nq        -1.5072               0.6952   
                  squad      nq        -0.6609               0.7910   
                  triviaqa   nq        -0.9437               0.7531   
bart_base_pt_long amalgam    nq        -0.2672               0.8665   
                  nq         nq        -0.2507               0.8706   
                  quac       nq        -0.9909               0.7599   
                  squad      nq        -0.6329               0.7929   
                  triviaqa   nq        -0.7363               0.7768   

                                        bertscore-recall  bertscore-f1  meteor  
base_model        trained_on tested_on                                          
T5_base_pt_long   amalgam    nq                   0.8533        0.8564  0.4940  
                  nq         nq                   0.8523        0.8566  0.4916  
                  quac       nq                   0.6813        0.6880  0.0761  
                  squad      nq                   0.8026        0.7964  0.3519  
                  triviaqa   nq                   0.7892        0.7702  0.2916  
bart_base_pt_long amalgam    nq                   0.8626        0.8643  0.5235  
                  nq         nq                   0.8646        0.8674  0.5288  
                  quac       nq                   0.7420        0.7505  0.2244  
                  squad      nq                   0.8051        0.7986  0.3553  
                  triviaqa   nq                   0.8161        0.7956  0.3674

In [54]:
evaluation_db_df[evaluation_db_df['tested_on'] == 'quac'].groupby(['base_model', 'trained_on', 'tested_on']).mean()

rouge1  rouge2  rougeL  rougeLsum  \
base_model        trained_on tested_on                                      
T5_base_pt_long   amalgam    quac       0.2573  0.1026  0.2522     0.2522   
                  nq         quac       0.1539  0.0339  0.1503     0.1503   
                  quac       quac       0.1720  0.0411  0.1710     0.1710   
                  squad      quac       0.1898  0.0451  0.1837     0.1837   
                  triviaqa   quac       0.1099  0.0193  0.0989     0.0989   
bart_base_pt_long amalgam    quac       0.2381  0.0883  0.2337     0.2337   
                  nq         quac       0.1530  0.0346  0.1496     0.1496   
                  quac       quac       0.2867  0.1404  0.2827     0.2827   
                  squad      quac       0.1893  0.0466  0.1842     0.1842   
                  triviaqa   quac       0.1219  0.0256  0.1137     0.1137   

                                        bleurt  bertscore-precision  \
base_model        trained_on tested_on                                
T5_base_pt_long   amalgam    quac      -0.9355               0.7889   
                  nq         quac      -1.3609               0.6927   
                  quac       quac      -1.3705               0.7713   
                  squad      quac      -1.1584               0.7355   
                  triviaqa   quac      -1.3861               0.6817   
bart_base_pt_long amalgam    quac      -1.0444               0.7585   
                  nq         quac      -1.3413               0.6925   
                  quac       quac      -0.8082               0.7964   
                  squad      quac      -1.2148               0.7368   
                  triviaqa   quac      -1.3745               0.6814   

                                        bertscore-recall  bertscore-f1  meteor  
base_model        trained_on tested_on                                          
T5_base_pt_long   amalgam    quac                 0.7823        0.7851  0.2551  
                  nq         quac                 0.7008        0.6964  0.1113  
                  quac       quac                 0.7500        0.7601  0.1653  
                  squad      quac                 0.7751        0.7543  0.2096  
                  triviaqa   quac                 0.7362        0.7073  0.1497  
bart_base_pt_long amalgam    quac                 0.7842        0.7705  0.2502  
                  nq         quac                 0.7024        0.6972  0.1126  
                  quac       quac                 0.7902        0.7928  0.2881  
                  squad      quac                 0.7733        0.7540  0.2059  
                  triviaqa   quac                 0.7417        0.7097  0.1640

In [55]:
evaluation_db_df[evaluation_db_df['tested_on'] == 'squad'].groupby(['base_model', 'trained_on', 'tested_on']).mean()

rouge1  rouge2  rougeL  rougeLsum  \
base_model        trained_on tested_on                                      
T5_base_pt_long   amalgam    squad      0.4877  0.2688  0.4552     0.4552   
                  nq         squad      0.3576  0.1441  0.3301     0.3301   
                  quac       squad      0.1800  0.0253  0.1734     0.1734   
                  squad      squad      0.4842  0.2684  0.4521     0.4521   
                  triviaqa   squad      0.2938  0.1059  0.2426     0.2426   
bart_base_pt_long amalgam    squad      0.4933  0.2793  0.4586     0.4586   
                  nq         squad      0.3490  0.1390  0.3226     0.3226   
                  quac       squad      0.3062  0.1133  0.2909     0.2909   
                  squad      squad      0.5034  0.2903  0.4692     0.4692   
                  triviaqa   squad      0.3507  0.1418  0.3104     0.3104   

                                        bleurt  bertscore-precision  \
base_model        trained_on tested_on                                
T5_base_pt_long   amalgam    squad     -0.3265               0.8691   
                  nq         squad     -0.7626               0.8008   
                  quac       squad     -1.3550               0.7693   
                  squad      squad     -0.3187               0.8714   
                  triviaqa   squad     -0.8792               0.7839   
bart_base_pt_long amalgam    squad     -0.3081               0.8690   
                  nq         squad     -0.7647               0.7975   
                  quac       squad     -0.8318               0.8332   
                  squad      squad     -0.2669               0.8752   
                  triviaqa   squad     -0.6454               0.8098   

                                        bertscore-recall  bertscore-f1  meteor  
base_model        trained_on tested_on                                          
T5_base_pt_long   amalgam    squad                0.8563        0.8621  0.4530  
                  nq         squad                0.7729        0.7863  0.2593  
                  quac       squad                0.7168        0.7417  0.1343  
                  squad      squad                0.8598        0.8649  0.4555  
                  triviaqa   squad                0.7999        0.7910  0.2894  
bart_base_pt_long amalgam    squad                0.8579        0.8629  0.4601  
                  nq         squad                0.7711        0.7838  0.2520  
                  quac       squad                0.7759        0.8029  0.2456  
                  squad      squad                0.8651        0.8695  0.4753  
                  triviaqa   squad                0.8252        0.8168  0.3382

In [56]:
evaluation_db_df[evaluation_db_df['tested_on'] == 'triviaqa'].groupby(['base_model', 'trained_on', 'tested_on']).mean()

rouge1  rouge2  rougeL  rougeLsum  \
base_model        trained_on tested_on                                      
T5_base_pt_long   amalgam    triviaqa   0.4126  0.1946  0.3468     0.3468   
                  nq         triviaqa   0.3140  0.1167  0.2731     0.2731   
                  quac       triviaqa   0.1248  0.0128  0.1162     0.1162   
                  squad      triviaqa   0.3098  0.1093  0.2667     0.2667   
                  triviaqa   triviaqa   0.3780  0.1613  0.3153     0.3153   
bart_base_pt_long amalgam    triviaqa   0.4357  0.2214  0.3687     0.3687   
                  nq         triviaqa   0.3319  0.1285  0.2864     0.2864   
                  quac       triviaqa   0.2188  0.0626  0.1978     0.1978   
                  squad      triviaqa   0.2978  0.1101  0.2603     0.2603   
                  triviaqa   triviaqa   0.4340  0.2187  0.3678     0.3678   

                                        bleurt  bertscore-precision  \
base_model        trained_on tested_on                                
T5_base_pt_long   amalgam    triviaqa  -0.6840               0.8371   
                  nq         triviaqa  -1.0624               0.7917   
                  quac       triviaqa  -1.5220               0.7418   
                  squad      triviaqa  -0.9154               0.8141   
                  triviaqa   triviaqa  -0.7718               0.8257   
bart_base_pt_long amalgam    triviaqa  -0.6265               0.8438   
                  nq         triviaqa  -1.0174               0.7999   
                  quac       triviaqa  -1.1692               0.8000   
                  squad      triviaqa  -0.9087               0.8152   
                  triviaqa   triviaqa  -0.6314               0.8444   

                                        bertscore-recall  bertscore-f1  meteor  
base_model        trained_on tested_on                                          
T5_base_pt_long   amalgam    triviaqa             0.8036        0.8191  0.3270  
                  nq         triviaqa             0.7388        0.7639  0.1860  
                  quac       triviaqa             0.6726        0.7050  0.0889  
                  squad      triviaqa             0.7652        0.7881  0.2255  
                  triviaqa   triviaqa             0.7966        0.8101  0.2995  
bart_base_pt_long amalgam    triviaqa             0.8113        0.8264  0.3546  
                  nq         triviaqa             0.7446        0.7708  0.2008  
                  quac       triviaqa             0.7167        0.7555  0.1509  
                  squad      triviaqa             0.7642        0.7881  0.2191  
                  triviaqa   triviaqa             0.8098        0.8259  0.3510